## This notebook generates an ics file

For some reason the kind folks that run my kids color guard program think it is best to have a schedule in google sheets rather than in say, I don't know.... Maybe a calendar. 

No problem. This fixes that. 

In [3]:
import pandas as pd
from ics import Calendar, Event
import pytz
import re
from datetime import timedelta, datetime
import sqlite3


# grab the data
sql = 'select school, datestamp, description, url from lunches where datestamp is not null order by school, datestamp;'
results = []
with sqlite3.connect('lunch.sqlite') as conn:
    cursor = conn.cursor()
    cursor.execute( sql )
    results = cursor.fetchall()

print( len(results))

111


In [4]:
# use this to turn text months into number months
months = {
    'January': 1, 'February': 2, 'March': 3,
    'April': 4, 'May': 5, 'June': 6,
    'July': 7, 'August': 8, 'September': 9,
    'October': 10, 'November': 11, 'December': 12
}

In [5]:
# take the calendar items and build an ical object then write it out to the file
cal = Calendar()

# constants for the description
information=r'https://www.avongrove.org/departments/food-services/school-menus'

# grab when i was last run so i can see in the calendar if its working
#last_updated_str = str( datetime.now( pytz.timezone('US/Eastern') ) )

for row in results:
    e = Event()
    
    # grab records for easy reference
    school = row[0].upper()
    date = row[1]
    description = row[2]
    url = row[3]
    month = None
    day = None
    year = None
        
    # parse out the date
    match = re.search(r'(\d\d\d\d)-(\d+)-(\d+)', date)
    if match:
        month = int(match.group(2))
        day = int(match.group(3))
        year = int(match.group(1))
    del match
    
    # build the event info
    e.name = school + ' daily menus\n'
    e.begin = datetime(year, month, day, 11, 0, 0, tzinfo=pytz.timezone('US/Eastern')) - timedelta(minutes=15)

    e.end = datetime(year, month, day, 11, 30, 0, tzinfo=pytz.timezone('US/Eastern')) 
    e.location = 'Avongrove: ' + school
    #e.url = url
    e.description = school + ' daily menu:\n\n' + description + '\n\nLink to PDF:\n' + url + '\n\nGeneral Information:\n' + information
    #e.organizer = 'bob@ruddy.net'
    
    # add event to the calendar
    cal.events.add(e)

# write out to an ics file
ics_file = 'lunch.ics'
with open(ics_file, 'w') as my_file:
    my_file.writelines(cal)

# run this to copy to server when finished
    scp ~ruddy/src/school_lunch/lunch.ics elm:/data/htdocs/elm/lunch.ics